## CSE 4705: Assignment 02 - Arad to Bucharest - BFS, DFS, UCS, GBFS, A* 


## Problem 1
[100] Write a routine that solves the problem of finds a travel path of cities from from Arad to Bucharest in Romania, as discussed in class. Do this using each of the following approaches (points shown in brackets):

1. [15] Breadth First Search (BFS)
2. [10] Depth First Search (DFS)
3. [25] Uniform Cost Search (UCS)
4. [25] Greedy Best First Search (GBFS)
5. [25] A*


You will use the map from Lecture 03 - Informed Search which shows the major cities in Romania and the distances between them for those cities that are directly connected.  Also, you will use the straight-line-distances shown in the adjacent table for your heuristic function, $h(n)$ for GBFS and A*.  A screenshot of the relevant slide is given below.  Data structures that store this information, romania_map and sld_to_bucharest, have been provided so you can access and apply this data in your algorithm implementations.  Details of these data structures are given below.


### Output for Each Routine

Each of your routines should return an output or set of outputs that clearly indicates the following:

1. The sequence of cities from Arad to Bucharest.  (Make sure the cities, Arad and Bucharest are explicitly listed as the first and last cities in your output.)  One suggestion is to return this output in the form of a list.
2. Cost to travel to each city from its predecessor.  
3. Total cost for the path.  

In the case of A* and Uniform Cost Search, your routines should return the *cheapest path*.  However, that will not necessarily be the case for BFS, DFS, or GBFS.  (Why not?)

### Romania Graph

You will use the data structure stored in the romania_map, assigned below to implement the search across the various cities to find a path from Arad to Bucharest.

Some details about romania_map:
- A dictionary of dictionaries 
- The outer dictionary is as follows: each key is a city and the value for that city is a nested dictionary of cities to which the said city is directly connected.
- The nested dictionary contains the cities to which the parent key is directly connected (keys) and the corresponding distances from the parent city to those respective cities (values).
- For example, for the city Oradea, we have a key in the outer dictionary (Oradea), and the associated value is a dictionary containing the Zerind and Sibiu as keys, where for each of these the values are the distances from Oradea to these respective cities.



In [1]:

romania_map = {
    'Oradea':{'Zerind':71, 'Sibiu':151},
    'Zerind':{'Oradea':71, 'Arad':75},
    'Arad':{'Zerind':75, 'Sibiu':140, 'Timisoara':118},
    'Timisoara':{'Arad':118, 'Lugoj':111},
    'Lugoj':{'Timisoara':111, 'Mehadia':70},
    'Mehadia':{'Lugoj':70, 'Dobreta':75},
    'Dobreta':{'Mehadia':75, 'Craiova':120},
    'Sibiu':{'Oradea':151, 'Fagaras':99, 'Rimnicu Vilcea':80, 'Arad':140},
    'Rimnicu Vilcea':{'Sibiu':80, 'Pitesti':97, 'Craiova':146},
    'Craiova':{'Rimnicu Vilcea':146, 'Pitesti':138, 'Dobreta':120},
    'Fagaras':{'Sibiu':99, 'Bucharest':211},
    'Pitesti':{'Rimnicu Vilcea':97, 'Bucharest':101, 'Craiova':138},
    'Neamt':{'Iasi':87},
    'Giurgiu':{'Bucharest':90},
    'Bucharest':{'Pitesti':101, 'Fagaras':211, 'Urziceni':85, 'Giurgiu':90},
    'Iasi':{'Neamt':87, 'Vaslui':92},
    'Urziceni':{'Bucharest':85, 'Vaslui':142, 'Hirsova':98},
    'Vaslui':{'Iasi':92, 'Urziceni':142},
    'Hirsova':{'Urziceni':98, 'Eforie':86},
    'Eforie':{'Hirsova':86}
}

### Heuristic Function Data - Straight-Line Distances to Bucharest

You will use the dictionary below as your resource for retrieving straight-line distance data for implementing the GBFS and A* algorithms.

In [2]:
sld_to_Bucharest = {'Arad':366,
                    'Bucharest':0,
                    'Craiova':160,
                    'Dobreta':242,
                    'Eforie':161,
                    'Fagaras':176,
                    'Giurgiu':77,
                    'Hirsova':151,
                    'Iasi':226,
                    'Lugoj':244,
                    'Mehadia':241,
                    'Neamt':234,
                    'Oradea':380,
                    'Pitesti':100,
                    'Rimnicu Vilcea':193,
                    'Sibiu':253,
                    'Timisoara':329,
                    'Urziceni':80,
                    'Vaslui':199,
                    'Zerind':374
                   }

### General usage code

##### Node class definition

In [3]:
class Node:
    def __init__(self, city,children = None, predecessor = None, weight = 0):
        self.city = city
        self.predecessor = predecessor
        self.children = children
        self.weight = weight
        self.agg_weight = weight
    
    def unvisited_adjacent_nodes(self, visited_nodes):
        unv_nodes = []
        for node in self.children:
            if node.city not in visited_nodes:
                    unv_nodes.append(node)
        return unv_nodes
    
    def is_goal_state(self, goal_city):
        return goal_city == self.city
    
    def set_children(self):
        children = []
        if self.city in romania_map.keys():
            for k,i in romania_map[self.city].items():
                children.append(Node(k,weight=i))
        self.children = children
 
    def return_path(self):
        total_cost = self.weight
        pred = self.predecessor
        
        ret_path = [f"{self.city} - {self.weight}km"]
        while pred is not None:
            total_cost += pred.weight
   
            if not pred.predecessor:
                ret_path.insert(0,pred.city)
            else:
                path_node = f"{pred.city} - {str(pred.weight)}km"
                ret_path.insert(0,path_node)
                
            pred = pred.predecessor
        
        ret_path.append(f'Total cost: {total_cost}km')
            
        return ret_path
    
    def update_frontier(self, frontier, queue_lookup,f_of_n):
        if self.city in queue_lookup.keys():
            existing_node = queue_lookup[self.city]
            if f_of_n['compare'](self, existing_node):
                existing_node.remove_from_priority_queue(frontier, queue_lookup)   
            else:  
                return
        
        frontier.put((self.agg_weight, self))
        queue_lookup[self.city] = self
        
    def remove_from_priority_queue(self,pq,queue_lookup):
        temp_list = []

        while not pq.empty():
            priority, element = pq.get()
            if element != self:
                temp_list.append((priority, element))
            else:
                del queue_lookup[self.city]
        
        for item in temp_list:
            pq.put(item)


##### Uninformed search algorithm

In [4]:
def uninformed_sa(start_node: Node, goal_city, data_s):
   visited_nodes = {}
   frontier = data_s
   error_msg = "Please make sure to introduce a valid starting point and destination "

   start_node.set_children()
   visited_nodes[start_node.city] = start_node
   frontier.append(start_node)
   
   if not goal_city in romania_map:
      return error_msg
   
   while frontier:
      
      if type(frontier)==list:
         current_node = frontier.pop(0)
      else:
         current_node = frontier.pop()
         
      for neighbor_node in current_node.unvisited_adjacent_nodes(visited_nodes):

         neighbor_node.predecessor = current_node
         
         if neighbor_node.is_goal_state(goal_city):
            return neighbor_node.return_path()
         neighbor_node.set_children()
         visited_nodes[neighbor_node.city] = neighbor_node
         frontier.append(neighbor_node)
   
   return error_msg

##### Informed search algorithm + UCS

In [5]:
from queue import PriorityQueue
def ucs_and_informed_s_algs(start_node: Node, goal_city, f_of_n):
    visited_nodes = {}
    frontier = PriorityQueue()
    queue_lookup = {}
    error_msg = "Please make sure to introduce a valid starting point and destination "

    if not goal_city in romania_map:
        return error_msg
    start_node.set_children()

    visited_nodes[start_node.city] = start_node
    frontier.put((f_of_n["value"](start_node), start_node))

    while not frontier.empty():
        current_node = frontier.get()[1]
        current_cost = current_node.agg_weight
        visited_nodes[current_node.city] = current_node
        if current_node.is_goal_state(goal_city):
            return current_node.return_path()

        current_node.set_children()

        neighbor_nodes = current_node.unvisited_adjacent_nodes(visited_nodes)
        for node in neighbor_nodes:
            node.agg_weight += current_cost
            node.predecessor = current_node
            node.update_frontier(frontier,queue_lookup,f_of_n)
        
    return error_msg

### 1. BFS Implementation

Provide your implementation of the BFS Search below.

In [6]:
uninformed_sa( Node('Arad'),'Bucharest',list())

['Arad',
 'Sibiu - 140km',
 'Fagaras - 99km',
 'Bucharest - 211km',
 'Total cost: 450km']

### 2.  DFS Implementation

Provide your implementation of the DFS Search below.

In [7]:
from collections import deque
uninformed_sa( Node('Arad'),'Bucharest',deque())

['Arad',
 'Timisoara - 118km',
 'Lugoj - 111km',
 'Mehadia - 70km',
 'Dobreta - 75km',
 'Craiova - 120km',
 'Pitesti - 138km',
 'Bucharest - 101km',
 'Total cost: 733km']

### 3. UCS Implementation

Provide your implementation of the UCS Search below.

In [8]:
def ucs(start_node: Node, goal_city):
    def f_n_value(node):
        return node.agg_weight
    def f_n_compare(node, node_b):
        return f_n_value(node) < f_n_value(node_b)
    
    f_n = dict(value = f_n_value,
               compare = f_n_compare)
    return ucs_and_informed_s_algs(start_node, goal_city,f_n)

In [9]:
ucs(Node('Arad'), 'Bucharest')

['Arad',
 'Sibiu - 140km',
 'Rimnicu Vilcea - 80km',
 'Pitesti - 97km',
 'Bucharest - 101km',
 'Total cost: 418km']

### 4. GBFS Implementation

Provide your implementation of the GBFS Search below.

In [10]:
def gbfs(start_node: Node, goal_city):
    def f_n_value(node):
        return sld_to_Bucharest[node.city]
    def f_n_compare(node, node_b):
        return False
    
    f_n = dict(value = f_n_value,
               compare = f_n_compare)
    return ucs_and_informed_s_algs(start_node, goal_city,f_n)

In [11]:
gbfs(Node('Arad'), 'Bucharest')

['Arad',
 'Sibiu - 140km',
 'Fagaras - 99km',
 'Bucharest - 211km',
 'Total cost: 450km']

### 5. A* Implementation

Provide your implementation of the A* Algorithm below.

In [12]:
def a_star(start_node: Node, goal_city):
    def f_n_value(node):
        return sld_to_Bucharest[node.city] + node.agg_weight
    def f_n_compare(node, node_b):
        return f_n_value(node) < f_n_value(node_b)
    
    f_n = dict(value = f_n_value,
               compare = f_n_compare)
    return ucs_and_informed_s_algs(start_node, goal_city,f_n)

In [13]:
a_star(Node('Arad'), 'Bucharest')

['Arad',
 'Sibiu - 140km',
 'Rimnicu Vilcea - 80km',
 'Pitesti - 97km',
 'Bucharest - 101km',
 'Total cost: 418km']